In [5]:
import os
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd

In [6]:
import dotenv as dtenv
dtenv.load_dotenv()

True

In [ ]:
# Initialize Spark Session with PostgreSQL JDBC driver
spark = SparkSession.builder \
    .appName("PostgreSQL-Spark Connection") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0") \
    .config("spark.executor.memory", "1g") \
    .config("spark.driver.memory", "1g") \
    .config("spark.executor.cores", "1") \
    .config("spark.driver.extraJavaOptions", "-Dlog4j.configuration=file:///opt/spark/conf/log4j.properties") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .getOrCreate()

# Check Spark version
print(f"Spark version: {spark.version}")

In [8]:
pg_host = "db"
pg_port = "5432"
pg_database = "my_database"
pg_user = "admin"
pg_password = "a8a7237dnaad"

jdbc_url = f"jdbc:postgresql://{pg_host}:{pg_port}/{pg_database}"

connection_properties = {
    "user": pg_user,
    "password": pg_password,
    "driver": "org.postgresql.Driver",
    "ssl": "false"
}

print(f"JDBC URL: {jdbc_url}")
print("Connection properties configured (password hidden)")

JDBC URL: jdbc:postgresql://db:5432/my_database
Connection properties configured (password hidden)


In [9]:
main_df = spark.read.jdbc(url=jdbc_url, table="ddn_mapa", properties=connection_properties)

In [10]:
type(main_df)

pyspark.sql.classic.dataframe.DataFrame

In [11]:
pd_df = main_df.toPandas()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:95: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  [PACKAGE_NOT_INSTALLED] PyArrow >= 11.0.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)
25/10/20 17:35:10 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [12]:
pd_df.head(10)

,cd_id,cd_cliente,cd_id_equipamento,cd_id_implemento,cd_id_operacao,cd_id_operacao_cb,cd_id_operacao_parada,cd_id_equipe,cd_id_operador,cd_id_jornada,...,vl_placa_florestal,vl_angulo_roll,vl_vazao_ideal,vl_inoculante_litros_ha,vl_contagem_viagens,vl_profundidade,fg_origem_alteracao,vl_metros_mangueira_hidroroll,vl_velocidade_programada_hidroroll,fase_operacao
0,4119387739000076,76,12872192000076,-1,8698000076,5250015000076,8698000076,1042666000076,88960000076,-1,...,None,NaN,0.0,0.0,None,0.0,NaN,0.0,0.0,9
1,4119211917000076,76,12585736000076,-1,5250022000076,5250022000076,-1,1042126000076,57192911000076,-1,...,None,NaN,0.0,0.0,None,0.0,NaN,0.0,0.0,8
2,4119211932000076,76,12585736000076,-1,8698000076,5250022000076,8698000076,1042126000076,57192911000076,-1,...,None,NaN,0.0,0.0,None,0.0,NaN,0.0,0.0,8
3,4119220995000076,76,12872192000076,-1,8698000076,5250015000076,8698000076,1042666000076,88960000076,-1,...,None,NaN,0.0,0.0,None,0.0,NaN,0.0,0.0,9
4,4119232684000076,76,12872192000076,-1,8698000076,5250015000076,8698000076,1042666000076,88960000076,-1,...,None,NaN,0.0,0.0,None,0.0,NaN,0.0,0.0,9
5,4119261335000076,76,12872192000076,-1,8679000076,5250015000076,8679000076,1042666000076,88960000076,-1,...,None,NaN,0.0,0.0,None,0.0,NaN,0.0,0.0,9
6,4119243548000076,76,12872192000076,-1,8698000076,5250015000076,8698000076,1042666000076,88960000076,-1,...,None,NaN,0.0,0.0,None,0.0,NaN,0.0,0.0,9
7,4119243549000076,76,12872192000076,-1,8698000076,5250015000076,8698000076,1042666000076,88960000076,-1,...,None,NaN,0.0,0.0,None,0.0,NaN,0.0,0.0,9
8,4119273908000076,76,12872192000076,-1,8679000076,5250015000076,8679000076,1042666000076,88960000076,-1,...,None,NaN,0.0,0.0,None,0.0,NaN,0.0,0.0,9
9,4119283491000076,76,12872192000076,-1,5250015000076,5250015000076,-1,1042666000076,88960000076,-1,...,None,NaN,0.0,0.0,None,0.0,NaN,0.0,0.0,9


In [13]:
main_df.createOrReplaceTempView("ddn_mapa_view")

In [14]:
result = spark.sql("""
    SELECT COUNT(*)
    FROM ddn_mapa_view
    WHERE 1=1
""")

In [16]:
result = spark.sql("""
    SELECT COUNT(*)
    FROM ddn_mapa_view as dmv
    WHERE dmv.vl_tempo_segundos < 1800
""")

In [17]:
result.show()

+--------+
|count(1)|
+--------+
|    3834|
+--------+

